<a href="https://colab.research.google.com/github/daniel-de-oliveira-souza/transitanalytics/blob/main/SummaryGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
from datetime import datetime, timedelta
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import openpyxl

# Function to add week number to the DataFrame
def add_week_number_column_from_filename(df, week_number):
    df['WeekNumber'] = week_number[-7:-5]
    return df

# Function to add month to the DataFrame
def add_month_column_from_filename(df, filename):
    base_filename = os.path.splitext(os.path.basename(filename))[0]
    words = base_filename.split("_")
    if len(words) > 1:
        month = words[1]
        df['Month'] = month
    return df

# Function to add date column to the DataFrame
def add_date_column_from_filename(df, week_number, year=None):
    if year is None:
        year = datetime.now().year

    first_day_of_year = datetime(year, 1, 1)
    days_to_add = (7 - first_day_of_year.weekday()) % 7
    first_monday_of_year = first_day_of_year + timedelta(days=days_to_add)
    monday_date = first_monday_of_year + timedelta(weeks=week_number - 1)
    df['Date'] = monday_date.date()
    return df

# Function to get all .xlsx files in a directory
def get_xlsx_files(directory):
    return glob.glob(os.path.join(directory, '*.xlsx'))

# Function to read an Excel file
def read_xlsx_file(file_path):
    try:
        return pd.read_excel(file_path)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Function to process an Excel file
def process_xlsx_file(file_path):
    df = read_xlsx_file(file_path)
    if df is not None:
        if 'LIRR' in file_path:
            df = add_month_column_from_filename(df, file_path)
            df = df.rename(columns={'% StkOut': '%StockOut'})

            month_mapping = {
                'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
            }
            df['Month_Num'] = df['Month'].map(month_mapping)
        else:
            week_number = int(file_path[-7:-5])
            df = add_week_number_column_from_filename(df, file_path)
            df = add_date_column_from_filename(df, week_number)

        df = df.iloc[:-3]
        df = df.rename(columns={'% StockOut': '%StockOut'})

        if len(df) > 6:
            df.columns = df.columns.str.replace('Sys StkOut', 'Sys_StkOut')
    return df

# Function to process multiple Excel files
def process_xlsx_files(directory):
    dataframes = [process_xlsx_file(file) for file in get_xlsx_files(directory)]
    return pd.concat(dataframes, ignore_index=True)

# Main execution starts here
my_xlsx_dir = '/content/FastMovers'
df = process_xlsx_files(my_xlsx_dir)


# Determine column names based on DataFrame structure
sys_stkout_column_exists = "Sys_StkOut" if len(df.columns) == 7 else None
monthnumber_or_weeknumber = 'Month_Num' if 'Month_Num' in df.columns else 'WeekNumber'
wh_or_sr = 'WH' if 'WH' in df.columns else 'SR'
srdescription_or_whname = 'SR Description' if 'SR Description' in df.columns else 'WH Name'

# Print the determined values
print("MonthNumber or WeekNumber:", monthnumber_or_weeknumber)
print("WH or SR:", wh_or_sr)
print("SR Description or WH Name:", srdescription_or_whname)

# Set new index and rearrange columns if needed
df = df.set_index([wh_or_sr]).sort_values(by=[monthnumber_or_weeknumber, wh_or_sr])
if sys_stkout_column_exists:
    column_order = list(df.columns)
    column_order.remove(sys_stkout_column_exists)
    column_order.insert(3, sys_stkout_column_exists)
    df = df[column_order]

# Sort the DataFrame by the month numbers
df = df.sort_values(by=[monthnumber_or_weeknumber, wh_or_sr])
grouped_df = df.reset_index().set_index([monthnumber_or_weeknumber, wh_or_sr])

# Totals calculation and rearranging totals_df
columns_to_sum = ["FM Count", "InStock", "StockOut"]
if sys_stkout_column_exists:
    columns_to_sum.insert(2, "Sys_StkOut")

totals = df.groupby(monthnumber_or_weeknumber)[columns_to_sum].sum().round(2)
totals['%StockOut'] = (totals['StockOut'] / totals['FM Count']).round(3)
totals['Avg_%StockOut_Change'] = totals['%StockOut'].pct_change().round(3)
totals['Avg_%StockOut_Change'].fillna(0, inplace=True)

totals = totals.rename(columns={'%StockOut': 'Avg_%StockOut'})
totals['Avg_%StockOut'] = (totals['Avg_%StockOut'] * 100).round(2).astype(str) + '%'
totals['Avg_%StockOut_Change'] = (totals['Avg_%StockOut_Change'] * 100).round(2).astype(str) + '%'
totals_df = totals[['Date'] + [col for col in totals.columns if col != 'Date']] if 'Date' in totals.columns else totals

# Plotting
plt.figure(figsize=(18, 10))
plot = sns.barplot(
    x=srdescription_or_whname,
    y="%StockOut",
    hue=monthnumber_or_weeknumber,
    data=df[df[srdescription_or_whname] != "Total"],
    palette="viridis",
)

plot.set_title('Percentage StockOut by Warehouse' if wh_or_sr == 'WH' else 'Percentage StockOut by Storeroom', fontsize=16)
plot.set_xlabel('Warehouse' if wh_or_sr == 'WH' else 'Storeroom', fontsize=14)
plot.set_xticklabels(plot.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=10)

plot1_path = 'Percentage StockOut by Warehouse.png' if wh_or_sr == 'WH' else 'Percentage StockOut by Storeroom.png'
plt.savefig(plot1_path)

if totals_df['Avg_%StockOut_Change'].dtype == 'object':
    totals_df['Avg_%StockOut_Change'] = totals_df['Avg_%StockOut_Change'].str.rstrip('%').astype('float') / 100.0
    totals_df['Avg_%StockOut_Change'] = totals_df['Avg_%StockOut_Change'].round(2)

if len(totals_df) > 1:
    sns.set(style="darkgrid", palette="muted")
    plt.figure(figsize=(15, 7))
    sns.lineplot(x=totals_df.index.get_level_values(0), y='Avg_%StockOut_Change', data=totals_df, marker='o', color='teal', label='Average % StockOut Change')

    plt.title('Average % StockOut Change Over Time', fontsize=16, fontweight='bold')
    plt.xlabel(monthnumber_or_weeknumber, fontsize=14)
    plt.ylabel('Average % StockOut Change', fontsize=14)
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(fontsize='12')
    plt.tight_layout()

    plot2_path = 'avg_stockout_change_plot.png'
    plt.savefig(plot2_path)

# Extracting Entity Name from the first .xlsx file
paths = get_xlsx_files(my_xlsx_dir)
entity = paths[0].split('/')[3].split()[0]

# Creating .xlsx file with Cumulative Data and Summary Statistics by Week
report_date = datetime.now().strftime('%Y-%m-%d')

xlsx_file_path = f'{entity}_Cumulative.xlsx'
with pd.ExcelWriter(xlsx_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name=f'Cumulative Data - {report_date}', index=True)
    grouped_df.to_excel(writer, sheet_name='Grouped by Month or Week', index=True)
    totals_df.to_excel(writer, sheet_name='Summary Stats by Month or Week', index=True)

    workbook = writer.book
    for sheet in writer.sheets.values():
        for column in sheet.columns:
            max_length = max(len(str(cell.value)) for cell in column)
            adjusted_width = max(max_length, 12)
            sheet.column_dimensions[column[0].column_letter].width = adjusted_width

    seaborn_sheet1 = writer.book.create_sheet('Percentage StockOut')
    img1 = openpyxl.drawing.image.Image(plot1_path)
    seaborn_sheet1.add_image(img1, 'A1')

    if len(totals_df) > 1:
        seaborn_sheet2 = writer.book.create_sheet('Avg %StockOut Change Over Time')
        img2 = openpyxl.drawing.image.Image(plot2_path)
        seaborn_sheet2.add_image(img2, 'A1')

# Completion message
print(f'Report has been saved to {xlsx_file_path}')
